In [3]:

import requests
import pandas as pd
import json

headers = {'User-Agent': "mbambal@purdue.edu"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )
# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(), orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

In [4]:
# Assuming 'sic_code.csv' is in your current working directory
sic_mapping = pd.read_csv('/Users/mayankbambal/Desktop/10K_API/data/sic_codes.csv', delimiter='|')

# Convert CIK column to string and add leading zeros to make it 10 digits
sic_mapping['CIK'] = sic_mapping['CIK'].astype(str).str.zfill(10)
sic_mapping['SIC'] = sic_mapping['SIC'].fillna(0)
sic_mapping['SIC'] = sic_mapping['SIC'].astype(int)
sic_mapping['SIC'] = sic_mapping['SIC'].astype(str)

company_data = pd.merge(companyData, sic_mapping, left_on='cik_str', right_on='CIK', how='left')

In [5]:
sic_code = input("Enter SIC code: ")
user_start = int(input("Enter start period (YYYY): "))
user_end = int(input("Enter end period (YYYY): "))

In [6]:
import requests
import pandas as pd
import json
import time
import random

filtered_cik_list = company_data[company_data['SIC'] == sic_code]['cik_str'].tolist()

ciks = filtered_cik_list

all_data = []

for cik in ciks[:3]:  # Limiting to the first CIK for demonstration. Remove [:1] for all CIKs.
    try:
        response = requests.get(
            f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
            headers=headers
        )
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data_str = response.content.decode('utf-8')
        data_dict = json.loads(data_str)
        rows = []
        cik_val = data_dict.get('cik')
        entityName = data_dict.get('entityName')

        for taxonomy, fact_group in data_dict.get('facts', {}).items():
            for fact_name, fact_data in fact_group.items():
                label = fact_data.get('label')
                description = fact_data.get('description')
                units_dict = fact_data.get('units', {})
                for unit, records in units_dict.items():
                    for record in records:
                        row = {
                            'cik': cik_val,
                            'entityName': entityName,
                            'taxonomy': taxonomy,
                            'fact_name': fact_name,
                            'label': label,
                            'description': description,
                            'unit': unit
                        }
                        row.update(record)
                        rows.append(row)
        all_data.extend(rows)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for CIK {cik}: {e}")
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error processing data for CIK {cik}: {e}")

    # Add a random delay between 1 and 3 seconds before the next API call
    time.sleep(random.uniform(1, 2))

df = pd.DataFrame(all_data)

# Convert 'start' and 'end' columns to datetime (if they exist)
if 'start' in df.columns:
    df['start'] = pd.to_datetime(df['start'], errors='coerce')
if 'end' in df.columns:
    df['end'] = pd.to_datetime(df['end'], errors='coerce')

# Filter the DataFrame:
# 1. 'fy' is between user_start and user_end.
# 2. The difference between 'end' and 'start' is greater than 350 days.
# 3. The year of the 'end' date equals the fiscal year ('fy').
filtered_df = df[
    (df['fy'] >= user_start) &
    (df['fy'] <= user_end) &
    ((df['end'] - df['start']).dt.days > 350) &
    (df['end'].dt.year == df['fy'])
]

In [7]:
filtered_df.head()

,cik,entityName,taxonomy,fact_name,label,description,unit,end,val,accn,fy,fp,form,filed,frame,start
828,1018724,"AMAZON.COM, INC.",us-gaap,AdvertisingExpense,Advertising Expense,Amount charged to advertising expense for the ...,USD,2021-12-31,1.690000e+10,0001018724-22-000005,2021,FY,10-K,2022-02-04,NaN,2021-01-01
831,1018724,"AMAZON.COM, INC.",us-gaap,AdvertisingExpense,Advertising Expense,Amount charged to advertising expense for the ...,USD,2022-12-31,2.060000e+10,0001018724-23-000004,2022,FY,10-K,2023-02-03,NaN,2022-01-01
834,1018724,"AMAZON.COM, INC.",us-gaap,AdvertisingExpense,Advertising Expense,Amount charged to advertising expense for the ...,USD,2023-12-31,2.030000e+10,0001018724-24-000008,2023,FY,10-K,2024-02-02,NaN,2023-01-01
836,1018724,"AMAZON.COM, INC.",us-gaap,AdvertisingExpense,Advertising Expense,Amount charged to advertising expense for the ...,USD,2024-12-31,2.140000e+10,0001018724-25-000004,2024,FY,10-K,2025-02-07,CY2024,2024-01-01
918,1018724,"AMAZON.COM, INC.",us-gaap,AllocatedShareBasedCompensationExpense,"Share-based Payment Arrangement, Expense",Amount of expense for award under share-based ...,USD,2021-12-31,1.275700e+10,0001018724-22-000005,2021,FY,10-K,2022-02-04,NaN,2021-01-01
